In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import matplotlib.colors as colors
from matplotlib.patches import Ellipse
%matplotlib inline
import math
import fitClus2D as fc
from scipy.stats import sigmaclip
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier

import astropy
from astropy.stats import sigma_clip
from astropy.io import fits
from astropy.wcs import WCS

import pandas as pd

cluster = 'm0416'

In [2]:
# Get galaxy data, make arrays for known cluster/noncluster members, 
data = pd.read_csv("Files/" + cluster+'_master.csv')
unknown = []
clusterarr = []
ncluster = []
clustz = 0.396
cut = 0.03

for i, row in data.iterrows():
    if not np.isnan(row['master_mag606']) and not np.isnan(row['master_mag814']) and row['master_d'] < 120 and row['master_mag814'] < 23.5 and row['master_valid'] == 1:
        if np.isnan(row['master_z']):
            unknown.append(row)
        else:
            if abs(row['master_z'] - clustz) <= cut:
                clusterarr.append(row)
            else:
                ncluster.append(row)

#Clip redshifts
clusterz = np.array([row['master_z'] for row in clusterarr])
arr, low, high = sigmaclip(clusterz, 3, 3)
print(low)
tmpclust = []

for row in clusterarr:
    if row['master_z'] > low and row['master_z'] < high:
        tmpclust.append(row)
    else:
        ncluster.append(row)
clusterarr = tmpclust

#Create feature and category arrays for knowns, and feature array for unknowns
XclustCM= [[row['master_mag814'], row['master_mag606'] - row['master_mag814']] for row in clusterarr]
XnclustCM = [[row['master_mag814'], row['master_mag606'] - row['master_mag814']] for row in ncluster]
Xcm = np.array(XclustCM + XnclustCM)

unknownXcm = [[row['master_mag814'], row['master_mag606'] - row['master_mag814']] for row in unknown]
unknownXcm = np.array(unknownXcm)

clustnum = len(clusterarr)
nclustnum = len(ncluster)

y = np.zeros(clustnum + nclustnum)
y[:len(clusterarr)] = 1

print(clustnum)

C:\Users\Jackson Lee\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


0.38350230017652603
145


In [8]:

test = sigma_clip(clusterz, maxiters = None)
for i, val in enumerate(test.mask):
    if val:
        print(clusterz[i])

0.3796
0.4239


In [6]:
datapaper = pd.read_csv("Files/" + cluster + "paper.csv")
papercluster = []
for i, row in datapaper.iterrows():
    if row.Status == "member-spec":
        papercluster.append(row)
        

included = [False for i in range(clustnum)]
used = [False for i in range(len(papercluster))]


for i, gal in enumerate(papercluster):
    for j, gal2 in enumerate(clusterarr):
        if abs(gal.RA - gal2.master_RA) <= 0.000001 and abs(gal.Dec - gal2.master_Dec) <= 0.000001:
            included[j] = True
            used[i] = True
            
for i, gal in enumerate(papercluster):
    if not used[i]:
        print(gal)
        

RA              64.00911
Dec           -24.088767
F606W_mag          23.75
F606W_err           0.02
F814W_mag          23.12
F814W_err           0.02
z                   0.38
Status       member-spec
Refs             1,5,6,7
Name: 34, dtype: object


In [60]:
def sigmaCut(X, y, p, k):
    clustermag = [dat[0] for i, dat in enumerate(X) if y[i] == 1]
    clustercolor = [dat[1] for i, dat in enumerate(X) if y[i] == 1]
    
    model = np.polyfit(clustermag, clustercolor, 1)
    predict = np.poly1d(model)
    
    sigma = np.std(clustercolor)
    
    probs = np.zeros(len(p))
    classes = np.zeros(len(p))
    
    nclust = 0
    ntot = 0
    
    for i, point in enumerate(X):
        if abs(point[1] - predict(point[0])) < k * sigma:
            if y[i] == 1:
                nclust += 1
            ntot += 1
    prob = nclust / ntot
    
    for i, point in enumerate(p):
        if abs(point[1] - predict(point[0])) < k * sigma:
               probs[i] = prob
               classes[i] = 1
    
    print(predict(23.3242) - sigma)
    return probs, classes

In [61]:
sigmaclass = sigmaCut(Xcm, y, unknownXcm, 1.0)[1]
print(np.sum(sigmaclass))

paperphot = []
for i, row in datapaper.iterrows():
    if row.Status == "member-phot":
        paperphot.append(row)

vis = [False for i in range(len(paperphot))]

for i, gal in enumerate(unknown):
    if sigmaclass[i] == 1:
        for j, gal2 in enumerate(paperphot):
            if abs(gal2.RA - gal.master_RA) <= 0.000001 and abs(gal2.Dec - gal.master_Dec) <= 0.000001:
                vis[j] = True
                sigmaclass[i] = True

for i, gal in enumerate(paperphot):
    if not vis[i]:
        print(gal)
            
        

0.5836854959511864
162.0
RA              3.584027
Dec           -30.375176
F606W_mag          23.91
F606W_err           0.01
F814W_mag          23.32
F814W_err            0.0
z                   -1.0
Status       member-phot
Refs                 4,5
Name: 172, dtype: object
